# MSHTA Suspicious Execution 01
Detection for mshta.exe suspicious execution patterns sometimes involving file polyglotism

## Rule Content
```
- title: MSHTA Suspicious Execution 01
  id: cc7abbd0-762b-41e3-8a26-57ad50d2eea3
  status: experimental
  description: Detection for mshta.exe suspicious execution patterns sometimes involving
    file polyglotism
  date: 22/02/2019
  modified: 22/02/2019
  author: Diego Perez (@darkquassar)
  references:
  - http://blog.sevagas.com/?Hacking-around-HTA-files
  - https://0x00sec.org/t/clientside-exploitation-in-2018-how-pentesting-has-changed/7356
  - https://docs.microsoft.com/en-us/dotnet/standard/data/xml/xslt-stylesheet-scripting-using-msxsl-script
  - https://medium.com/tsscyber/pentesting-and-hta-bypassing-powershell-constrained-language-mode-53a42856c997
  tags:
  - attack.defense_evasion
  - attack.t1140
  logsource:
    category: process_creation
    product: windows
    service: null
  falsepositives:
  - False positives depend on scripts and administrative tools used in the monitored
    environment
  level: high
  detection:
    selection1:
      CommandLine:
      - '*mshta vbscript:CreateObject("Wscript.Shell")*'
      - '*mshta vbscript:Execute("Execute*'
      - '*mshta vbscript:CreateObject("Wscript.Shell").Run("mshta.exe*'
    selection2:
      Image:
      - C:\Windows\system32\mshta.exe
      CommandLine:
      - '*.jpg*'
      - '*.png*'
      - '*.lnk*'
      - '*.xls*'
      - '*.doc*'
      - '*.zip*'
    condition: selection1 or selection2

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:(*mshta\ vbscript\:CreateObject\(\"Wscript.Shell\"\)* OR *mshta\ vbscript\:Execute\(\"Execute* OR *mshta\ vbscript\:CreateObject\(\"Wscript.Shell\"\).Run\(\"mshta.exe*) OR (process_path:("C\:\\Windows\\system32\\mshta.exe") AND process_command_line.keyword:(*.jpg* OR *.png* OR *.lnk* OR *.xls* OR *.doc* OR *.zip*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()